## test MongoDB function

In [1]:
from src.dataread import read_file

In [2]:
!ls

LICENSE            app                yelp-api.ipynb     yelp_api_key.yaml
MongoDB.ipynb      data               yelp-ver1.ipynb    yelp_dataset
README.md          request_example.py yelp.ipynb         yelp_helpers.py
__pycache__        src                yelp.py


In [3]:
businesses = read_file('./yelp_dataset/yelp_academic_dataset_business.json')

In [5]:
businesses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 15 columns):
address         188593 non-null object
attributes      162807 non-null object
business_id     188593 non-null object
categories      188052 non-null object
city            188593 non-null object
hours           143791 non-null object
is_open         188593 non-null int64
latitude        188587 non-null float64
longitude       188587 non-null float64
name            188593 non-null object
neighborhood    188593 non-null object
postal_code     188593 non-null object
review_count    188593 non-null int64
stars           188593 non-null float64
state           188593 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 21.6+ MB


In [6]:
from pymongo import MongoClient